In [1]:
import numpy as np
import matplotlib.pyplot as plt 

%pylab inline
plt.style.use('seaborn-dark')
plt.rc('font', size=15)


import warnings
warnings.filterwarnings("ignore")

Populating the interactive namespace from numpy and matplotlib


In [2]:
def PrintTriangular(mas, i, lines=None):
    if lines is None:
        lines = len(mas)
        
    for line in range(lines):
        for n in range(line + 1 - i):
            print('{0:7.4f}'.format(mas[line][n]), end=' ')
        print()

In [149]:
def Interpolation(x, u, a):
    N = len(x) - 1
    U = np.zeros((N+1,N+1))
    U[:, 0] = u

    for k in range(1, N+1):   
        for n in range(N + 1 - k):
            U[n,k] = k * (U[n,k-1]-U[n+1,k-1]) / (x[n]-x[n+k])
            
    sm = 0.
    for n in range(N+1):
        P = 1.
        for k in range(n):
            P *= (a - x[k])  
            
        sm += U[0,n] / fact(n)*P
        
    return sm

# u - функция, a, b - границы ее задания, x0 - точка в которой береться производная
def _get_U(u, a, b, x0, N):
    x = np.linspace(a, b, N)
    
    U = np.zeros((N, N))
    U[:, 0] = u(x)
    for k in range(1, N):   
        for n in range(N - k):
            U[n,k] = k * (U[n,k-1]-U[n+1,k-1]) / (x[n]-x[n+k])
    
 
    index = [i for i in range(N)]
    n = min(index, key=lambda i: abs(x0 - x[i]))
    return U[0], n

# работает правильно, все как надо
def diff_1(u, a, b, x0, N):
    U, n = _get_U(u, a, b, x0, N)
    h = (b - a) / N
    
    # первая производная со 2-ым порядком точности
    ans = U[1] + U[2]*(x0-h/2)
    
    return ans
    
    
# а вот тут что то оочень странное
def diff_10(u, a, b, x0, N):
    U, n = _get_U(u, a, b, x0, N)
    h = (b - a) / N
    
    # первая производная со 2-ым порядком точности
    ans = (U[n-1] -2*U[n] + U[n+1]) / h**2
    
    return ans, n

diff_10(lambda x: x**2, 0, 2, 1, 8)

(-4.339639758654812e-12, 4)

In [125]:
def RicherdsonExtrapolation(func, a, b, x0, EPS=10, p=1, q=1, r=2, N=3, S=15):
    U = np.zeros((S,S))
    R = np.zeros((S,S))
    p_eff = np.zeros((S,S))

    s = 1
    U[0][0] = diff_1(func, a, b, x0, N)
    while True:
        U[s][0] = diff_1(func, a, b, x0, r**s*N)

        for n in range(s):
            R[s][n] = (U[s][n] - U[s-1][n]) / (r**(p + n*q) - 1)
            
            p_eff[s][n] = log(abs(R[s-1][n]/R[s][n])) / log(r)
            
            U[s][n + 1] = U[s][n] + R[s][n]
            
            if abs(R[s][n]) < (0.1)**EPS:
                return (U[s][n], R[s][n], p_eff[s][n], p*(s-1)+n*q | p, s, n, U, R, p_eff)

            
        
        s += 1
        if s > 13: 
            print('oops')
            return (U[s-1][s-1], R[s-1][s-1], p_eff[s-1][s-3], p + s*q, s, s, U, R, p_eff)



In [137]:
f = lambda x: 2*x**2 + 20

ans = RicherdsonExtrapolation(f, a=0, b=2, x0=1, EPS=5)
re = " \
Ответ: {0:12.10f}\n \
Погрешность: {1:12.10f}\n \
Эффективный порядок точности: {2:7.10f}\n \
Теоретический порядок точности: {3:7.5f}\n \
".format(ans[0], ans[1], ans[2], ans[3])
print(re)
PrintTriangular(ans[-3], 0, ans[4])
PrintTriangular(ans[-2], 1, ans[4])
PrintTriangular(ans[-1], 2, ans[4])

 Ответ: 3.9999964591
 Погрешность: 0.0000042491
 Эффективный порядок точности: 4.5698550001
 Теоретический порядок точности: 7.00000
 
 4.6667 
 4.1333  3.6000 
 4.0303  3.9273  4.0364 
 4.0072  3.9842  4.0032  3.9984 
 4.0018  3.9963  4.0003  3.9999  4.0000 

-0.5333 
-0.1030  0.1091 
-0.0231  0.0190 -0.0047 
-0.0055  0.0040 -0.0004  0.0001 


 2.3720 
 2.1598  2.5236 
 2.0747  2.2327  3.5546 
